In [ ]:
%%html
<style> 
    table {display: block;} 
</style>

# Bidirectional LSTM-CRF models for Sequence Tagging
## Zhiheng Huangm Wei Xu and Kai Yu
### Code
- https://github.com/guillaumegenthial/tf_ner/tree/master/models/lstm_crf
- https://github.com/sarveshsparab/BiLSTMCRFSeqTag

### Installing the required dependencies

In [ ]:
!pip install tensorflow
!pip install numpy

### Adding paths to sys paths

In [32]:
import sys
sys.path.append('../')
sys.path.append('../model')

### Supressing warning level messages in output

In [33]:
import warnings
warnings.filterwarnings('ignore')

### Instantiating an object of the NER parent class implementation 

In [34]:
from main import BiLSTMCRFSeqTag
from masked_conv import masked_conv1d_and_max
from ner import NER

blcst = BiLSTMCRFSeqTag()

### Instantiating the input files for the model
### 3 files required
- train : For the model to train
- dev : For the model to validate the training
- test : To evaluate the performance to the model

In [35]:
file_dict = dict()
file_dict['train'] = '../data/example/tester.txt'
file_dict['test'] = '../data/example/tester.txt'
file_dict['dev'] = '../data/example/tester.txt'

### Reading from the dataset
- Description
    * Reads a dataset in preparation for train or test. Returns data in proper format for train or test.
- Returns
    * A dictionary of file_dict keys as keys and values as lists of lines, where in each line is further tokenized on the column delimiter and extracted as a list
- Arguments

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Name</th>
            <th>Default</th>
            <th>Purpose</th>
            <th>Required</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan="2">Standard</td>
            <td>file_dict</td>
            <td>-</td>
            <td>A dictionary with input file locations</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>dataset_name</td>
            <td>CoNLL03</td>
            <td>Name of the dataset required for calling appropriate utils, converters</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>args</td>
            <td>N/A</td>
            <td>None</td>
            <td>N/A</td>
            <td>✖</td>
        </tr>
        <tr>
            <td rowspan="2">kwargs</td>
            <td>fileHasHeaders</td>
            <td>True</td>
            <td>Flag to check if input file has headers</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>columnDelimiter</td>
            <td>`space`</td>
            <td>Delimiter in the data input</td>
            <td>✖</td>
        </tr>
    </tbody>
</table>

In [36]:
data = blcst.read_dataset(file_dict, "CoNLL2003")

[main.py:151 - read_dataset() ] Invoked read_dataset method
[main.py:152 - read_dataset() ] With parameters : 
[main.py:153 - read_dataset() ] {'train': '../data/example/tester.txt', 'test': '../data/example/tester.txt', 'dev': '../data/example/tester.txt'}
[main.py:154 - read_dataset() ] CoNLL2003
[main.py:155 - read_dataset() ] ()
[main.py:156 - read_dataset() ] {}
[main.py:179 - read_dataset() ] Returning data : 
[main.py:180 - read_dataset() ] {'train': [['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['New', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['York', '-', '-', 'I-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Raph', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['London', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', 

### Extracting the ground truth data
- Description
    * Converts test data into common format for evaluation \[i.e. same format as predict()\] 
    * This added step/layer of abstraction is required due to the refactoring of read_dataset_train() and read_dataset_test() back to the single method of read_dataset() along with the requirement on the format of the output of predict() and therefore the input format requirement of evaluate()
- Returns
    * \[tuple,...\], i.e. list of tuples. \[SAME format as output of predict()\]
    * Each tuple is (start index, span, mention text, mention type)
    * Where:
         - start index: int, the index of the first character of the mention span. None if not applicable.
         - span: int, the length of the mention. None if not applicable.
         - mention text: str, the actual text that was identified as a named entity. Required.
         - mention type: str, the entity/mention type. None if not applicable.
- Arguments

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Name</th>
            <th>Default</th>
            <th>Purpose</th>
            <th>Required</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Standard</td>
            <td>data</td>
            <td>-</td>
            <td>data in proper format for train or test. [i.e. format of output from read_dataset]</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>args</td>
            <td>N/A</td>
            <td>None</td>
            <td>N/A</td>
            <td>✖</td>
        </tr>
        <tr>
            <td rowspan="4">kwargs</td>
            <td>wordPosition</td>
            <td>0</td>
            <td>Column number with the mention word</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>tagPosition</td>
            <td>3</td>
            <td>Column number with the entity tag</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>writeGroundTruthToFile</td>
            <td>True</td>
            <td>Flag to enable writing ground truths to a file</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>groundTruthPath</td>
            <td>../results/groundTruths.txt</td>
            <td>Location to save the ground truths file</td>
            <td>✖</td>
        </tr>
    </tbody>
</table>

In [37]:
groundTruth = blcst.convert_ground_truth(data, None)

[main.py:95 - convert_ground_truth() ] Invoked convert_ground_truth method
[main.py:96 - convert_ground_truth() ] With parameters : 
[main.py:97 - convert_ground_truth() ] {'train': [['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['New', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['York', '-', '-', 'I-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Raph', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['London', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Marie', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['Paris', '-', '-', 'B-LOC', '-', '-', '-

[main.py:98 - convert_ground_truth() ] (None,)
[main.py:99 - convert_ground_truth() ] {}
[main.py:109 - convert_ground_truth() ] Returning ground truths for the test input file :
[main.py:110 - convert_ground_truth() ] [[None, None, 'in', 'O'], [None, None, 'New', 'B-LOC'], [None, None, 'York', 'I-LOC'], [None, None, 'Raph', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'London', 'B-LOC'], [None, None, 'Marie', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'Paris', 'B-LOC'], [None, None, 'Dominik', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'Berlin', 'B-LOC'], [None, None, 'Raul', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'Mexico', 'B-LOC'], [None, None, 'Laure', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'Milan', 'B-LOC'], [None, None, 'Alexandr', 'B-PER'], [None, None, 'lives', 'O'], [None, None, 'in', 'O'], [None, None, 'Mosc

### Training the model
- Description
    * Trains he model on the parsed data
    * Calls the internal save_model method to save the trained model for predictions
- Returns
    * model   - Trained ELMo model object
    * sess    - Tensorflow session object (will be used to maintain the tensorflow session instance)
    * saver   - Tensorflow saver instance (will be used to load model again)
- Arguments

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Name</th>
            <th>Default</th>
            <th>Purpose</th>
            <th>Required</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Standard</td>
            <td>data</td>
            <td>-</td>
            <td>Parsed input data in the format returned by read_dataset method</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>args</td>
            <td>N/A</td>
            <td>None</td>
            <td>N/A</td>
            <td>✖</td>
        </tr>
        <tr>
            <td rowspan="20">kwargs</td>
            <td>dimChars</td>
            <td>100</td>
            <td>Model character level dimensionality</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>dim</td>
            <td>300</td>
            <td>Model dimensionality</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>dropout</td>
            <td>0.5</td>
            <td>Model dropout rate</td>
            <td>✖<td>
        </tr>
        <tr>
            <td>epochs</td>
            <td>25</td>
            <td>Number of epoch to run for training</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>batchSize</td>
            <td>20</td>
            <td>Training batch sizes</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>filterNum</td>
            <td>50</td>
            <td>Filter area size</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>lstmSize</td>
            <td>100</td>
            <td>State size of the Multi-LSTM layers</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>vocabWordsPath</td>
            <td>../dev/vocab.words.txt</td>
            <td>Location of the parsed words set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>vocabCharsPath</td>
            <td>../dev/vocab.chars.txt</td>
            <td>Location of the parsed charcters set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>vocabTagsPath</td>
            <td>../dev/vocab.tags.txt</td>
            <td>Location of the parsed tags set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>gloveCompressedNPZPath</td>
            <td>../dev/glove.npz</td>
            <td>Location of the extracted Glove embeddings from input data in compressed form</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>paramsPath</td>
            <td>../results/params.json</td>
            <td>Location where model parameters get saved</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>inputFileWordsPath</td>
            <td>../dev/{}.words.txt</td>
            <td>Location of the extracted words from the input files</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>inputFileTagsPath</td>
            <td>../dev/{}.tags.txt</td>
            <td>Location of the extracted tags from the input files</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>checkpointPath</td>
            <td>../results/checkpoint</td>
            <td>Location to save intermediate checkpoints</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>modelPath</td>
            <td>../results/saved_model</td>
            <td>Location to save the best model</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>gloveEmbedPath</td>
            <td>../resources/glove/glove.840B.300d.txt</td>
            <td>Location fo the glove embedding file</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>wordPosition</td>
            <td>0</td>
            <td>Column number with the mention word</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>tagPosition</td>
            <td>3</td>
            <td>Column number with the entity tag</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>writeInputToFile</td>
            <td>true</td>
            <td>Flag to toggle behaviour of the internal data_converter method</td>
            <td>✔</td>
        </tr>
    </tbody>
</table>

In [38]:
blcst.train(data, None)

[main.py:217 - train() ] Invoked train method
[main.py:218 - train() ] With parameters : 
[main.py:219 - train() ] {'train': [['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['New', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['York', '-', '-', 'I-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Raph', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['London', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Marie', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['Paris', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Dominik', '-

[main.py:220 - train() ] (None,)
[main.py:221 - train() ] {}
[main.py:466 - data_converter() ] Invoked data_converter method
[main.py:467 - data_converter() ] With parameters : 
[main.py:468 - data_converter() ] {'train': [['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['New', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['York', '-', '-', 'I-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Raph', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['London', '-', '-', 'B-LOC', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [], ['Marie', '-', '-', 'B-PER', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['lives', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['in', '-', '-', 'O', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], [

[main.py:469 - data_converter() ] (None,)
[main.py:470 - data_converter() ] {}
[main.py:472 - data_converter() ] Parsing input and building the corresponding data files
[main.py:475 - data_converter() ] Building vocab files
[main.py:526 - build_vocab_files() ] Build vocab words (may take a while)
[main.py:537 - build_vocab_files() ] - done. Kept 21 out of 21
[main.py:540 - build_vocab_files() ] Build vocab chars
[main.py:547 - build_vocab_files() ] - done. Found 32 chars
[main.py:554 - build_vocab_files() ] Build vocab tags (may take a while)
[main.py:562 - build_vocab_files() ] - done. Found 4 tags.
[main.py:479 - data_converter() ] Fetching glove embeddings from file : ../resources/glove/glove.840B.300d.txt
[main.py:573 - incorporate_glove_embeddings() ] Reading GloVe file (may take a while)
[main.py:577 - incorporate_glove_embeddings() ] - At line 0
[main.py:577 - incorporate_glove_embeddings() ] - At line 100000
[main.py:577 - incorporate_glove_embeddings() ] - At line 200000
[main

[builder_impl.py:414 - save() ] SavedModel written to: ../results/saved_model\temp-b'1556531404'\saved_model.pb
[main.py:428 - save_model() ] Model saved at location : ../results/saved_model


### Generating predictions
- Description
    * Parses and converts the input sentence provided in a file for predicting the NER tags
    * Calls the internal load_model method to load the trained model for predictions
- Returns
    * \[tuple,...\], i.e. list of tuples.
    * Each tuple is (start index, span, mention text, mention type)
    * Where:
         - start index: int, the index of the first character of the mention span. None if not applicable.
         - span: int, the length of the mention. None if not applicable.
         - mention text: str, the actual text that was identified as a named entity. Required.
         - mention type: str, the entity/mention type. None if not applicable.

         `NOTE: len(predictions) should equal len(data) AND the ordering should not change [important for evaluation. See note in evaluate() about parallel arrays.]`
- Arguments

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Name</th>
            <th>Default</th>
            <th>Purpose</th>
            <th>Required</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Standard</td>
            <td>data</td>
            <td>-</td>
            <td>The file location with the input text in the common format for prediction</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>args</td>
            <td>N/A</td>
            <td>None</td>
            <td>N/A</td>
            <td>✖</td>
        </tr>
        <tr>
            <td rowspan="11">kwargs</td>
            <td>fileHasHeaders</td>
            <td>True</td>
            <td>Flag to check if input file has headers</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>loadModelFrom</td>
            <td>checkpoint</td>
            <td>Flag to decide to choose to load model from 'checkpoint' or 'saved_model'</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>vocabWordsPath</td>
            <td>../dev/vocab.words.txt</td>
            <td>Location of the parsed words set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>vocabCharsPath</td>
            <td>../dev/vocab.chars.txt</td>
            <td>Location of the parsed charcters set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>vocabTagsPath</td>
            <td>../dev/vocab.tags.txt</td>
            <td>Location of the parsed tags set</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>gloveCompressedNPZPath</td>
            <td>../dev/glove.npz</td>
            <td>Location of the extracted Glove embeddings from input data in compressed form</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>paramsPath</td>
            <td>../results/params.json</td>
            <td>Location where model parameters get saved</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>checkpointPath</td>
            <td>../results/checkpoint</td>
            <td>Location to save intermediate checkpoints</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>modelPath</td>
            <td>../results/saved_model</td>
            <td>Location to save the best model</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>writePredsToFile</td>
            <td>True</td>
            <td>Flag to enable writing predictions to file</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>predsPath</td>
            <td>../results/predictions.txt</td>
            <td>Location where to write predictions into</td>
            <td>✖</td>
        </tr>
    </tbody>
</table>

In [39]:
predictions = blcst.predict("../data/sample/ner_test_input.txt", None, writeInputToFile=False)

[main.py:301 - predict() ] Invoked predict method
[main.py:302 - predict() ] With parameters : 
[main.py:303 - predict() ] ../data/sample/ner_test_input.txt
[main.py:304 - predict() ] (None,)
[main.py:305 - predict() ] {'writeInputToFile': False}
[main.py:431 - load_model() ] Invoked load_model method
[main.py:432 - load_model() ] With parameters : 
[main.py:433 - load_model() ] {'writeInputToFile': False}
[estimator.py:1739 - maybe_overwrite_model_dir_and_session_config() ] Using default config.
[estimator.py:201 - __init__() ] Using config: {'_model_dir': '../results/checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eva

### Evaluate the trained model
- Description
    * Calculates evaluation metrics on chosen benchmark dataset
        - Precision
        - Recall
        - F1 Score
- Returns
    * Tuple with metrics (p,r,f1). Each element is float.
- Arguments

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Name</th>
            <th>Default</th>
            <th>Purpose</th>
            <th>Required</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan="2">Standard</td>
            <td>predictions</td>
            <td>N/A</td>
            <td>List of predicted labels</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>groundTruths</td>
            <td>N/A</td>
            <td>List of ground truth labels</td>
            <td>✔</td>
        </tr>
        <tr>
            <td>args</td>
            <td>N/A</td>
            <td>None</td>
            <td>N/A</td>
            <td>✖</td>
        </tr>
        <tr>
            <td rowspan="2">kwargs</td>
            <td>predsPath</td>
            <td>../results/predictions.txt</td>
            <td>Location from where to read predictions from</td>
            <td>✖</td>
        </tr>
        <tr>
            <td>groundTruthPath</td>
            <td>../results/groundTruths.txt</td>
            <td>Location from where to read ground truths from</td>
            <td>✖</td>
        </tr>
    </tbody>
</table>

In [40]:
blcst.evaluate([col[3] for col in predictions], [col[3] for col in groundTruth], None)

[main.py:379 - evaluate() ] Invoked evaluate method
[main.py:380 - evaluate() ] With parameters : 
[main.py:381 - evaluate() ] ['B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'O', 'B-LOC', 'B-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-PER', 'B-LOC', 'B-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC']
[main.py:382 - evaluate() ] ['O', 'B-LOC', 'I-LOC', 'B-PER', 'O', 'O', 'B-LOC', 'B-PER', 'O', 'O', 'B-LOC', 'B-PER', 'O', 'O', 'B-LOC', 'B

(22.22222222222222, 47.05882352941176, 30.18867924528302)